In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext("local", "first app")

## Module

In [3]:
import math

In [4]:
class CFG : 
    sigma = 0.5

## Data Preprocessing

In [5]:
raw_fr = sc.textFile("facebook_combined.txt")

In [6]:
raw_fr.take(10)

['0 1', '0 2', '0 3', '0 4', '0 5', '0 6', '0 7', '0 8', '0 9', '0 10']

In [7]:
fr_split = raw_fr.map(lambda x : x.split(' '))

In [8]:
fr_split = fr_split.map(lambda x : (int(x[0]), int(x[1])))

In [9]:
fr_split.take(10)

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10)]

In [10]:
friends_list = fr_split.groupByKey().mapValues(list)

In [11]:
friends_list.take(10)

[(0,
  [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   136,
   137,
   138,

In [12]:
len(friends_list.collect())

3663

## 2way friends list

In [13]:
def map_func(input_data) : 
    rnum, vec = input_data
    
    tmp_list = []
    for element in vec : 
        tmp_list.append((rnum, element))
        tmp_list.append((element, rnum))
    
    return tmp_list

In [14]:
way2_friends_map = friends_list.flatMap(map_func)

In [15]:
way2_friends_list = way2_friends_map.groupByKey().mapValues(list)

In [16]:
way2_friends_list.take(10)

[(0,
  [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   136,
   137,
   138,

## Jaccard Similarity

In [17]:
broad_friends = sc.broadcast(way2_friends_list.collect())

In [18]:
def cal_jaccard(vec1, vec2) : 
    set1 = set(vec1)
    set2 = set(vec2)
    return len(set1.intersection(set2)) / len(set1.union(set2))

In [19]:
def make_result(input_data) : 
    friends_list = [broad_friends.value[num] for num in input_data[1]]
    tmp_list = []
    for vec1 in friends_list : 
        for vec2 in friends_list : 
            if (vec1[0] < vec2[0]) : 
                val = cal_jaccard(vec1[1], vec2[1])
                tmp_list.append(((vec1[0], vec2[0]), val))
    return tmp_list

In [20]:
tmp_result = way2_friends_list.flatMap(lambda x : make_result(x))

In [21]:
result = tmp_result.filter(lambda x : x[1] > CFG.sigma).sortBy(lambda x : x[0])

In [22]:
result.collect()

[((2, 14), 0.5625),
 ((2, 14), 0.5625),
 ((2, 14), 0.5625),
 ((2, 14), 0.5625),
 ((2, 14), 0.5625),
 ((2, 14), 0.5625),
 ((2, 14), 0.5625),
 ((2, 14), 0.5625),
 ((2, 14), 0.5625),
 ((2, 149), 0.6),
 ((2, 149), 0.6),
 ((2, 149), 0.6),
 ((2, 149), 0.6),
 ((2, 149), 0.6),
 ((2, 149), 0.6),
 ((2, 149), 0.6),
 ((2, 149), 0.6),
 ((2, 149), 0.6),
 ((2, 162), 0.8),
 ((2, 162), 0.8),
 ((2, 162), 0.8),
 ((2, 162), 0.8),
 ((2, 162), 0.8),
 ((2, 162), 0.8),
 ((2, 162), 0.8),
 ((2, 162), 0.8),
 ((2, 226), 0.6),
 ((2, 226), 0.6),
 ((2, 226), 0.6),
 ((2, 226), 0.6),
 ((2, 226), 0.6),
 ((2, 226), 0.6),
 ((2, 226), 0.6),
 ((2, 226), 0.6),
 ((2, 226), 0.6),
 ((4, 78), 0.7272727272727273),
 ((4, 78), 0.7272727272727273),
 ((4, 78), 0.7272727272727273),
 ((4, 78), 0.7272727272727273),
 ((4, 78), 0.7272727272727273),
 ((4, 78), 0.7272727272727273),
 ((4, 78), 0.7272727272727273),
 ((4, 78), 0.7272727272727273),
 ((4, 181), 0.8181818181818182),
 ((4, 181), 0.8181818181818182),
 ((4, 181), 0.8181818181818182

In [25]:
result.distinct().collect()

[((2, 14), 0.5625),
 ((2, 149), 0.6),
 ((2, 162), 0.8),
 ((2, 226), 0.6),
 ((4, 78), 0.7272727272727273),
 ((4, 181), 0.8181818181818182),
 ((4, 195), 0.7272727272727273),
 ((4, 218), 0.7272727272727273),
 ((4, 273), 0.7272727272727273),
 ((4, 275), 0.8181818181818182),
 ((4, 306), 0.7272727272727273),
 ((4, 328), 0.7272727272727273),
 ((6, 89), 0.5555555555555556),
 ((6, 147), 0.7142857142857143),
 ((6, 319), 0.5555555555555556),
 ((6, 327), 0.6666666666666666),
 ((7, 339), 0.5161290322580645),
 ((8, 91), 0.7777777777777778),
 ((8, 259), 0.7777777777777778),
 ((9, 26), 0.5432098765432098),
 ((9, 188), 0.5909090909090909),
 ((11, 12), 1.0),
 ((11, 15), 1.0),
 ((11, 18), 1.0),
 ((11, 37), 1.0),
 ((11, 43), 1.0),
 ((11, 74), 1.0),
 ((11, 114), 1.0),
 ((11, 209), 1.0),
 ((11, 210), 1.0),
 ((11, 215), 1.0),
 ((11, 287), 1.0),
 ((11, 292), 1.0),
 ((11, 335), 1.0),
 ((12, 15), 1.0),
 ((12, 18), 1.0),
 ((12, 37), 1.0),
 ((12, 43), 1.0),
 ((12, 74), 1.0),
 ((12, 114), 1.0),
 ((12, 209), 1.0),
